# **Backgammon MCTS/NN** 

## Introduction:
We are aiming to create an AI capable of playing backgammon at a high level. To do so we will implement a NN backed MCTS algorithm. 

## Methodology:

1.   Make backgammon game with board state information as array
2.   Implement MCTS with a default heuristic
3.   Simulate thousands of games backgammon (do we update the networks or just play random games? TD-Gammon used the networks to create training data)
4.   Train NN 


##Resources:
[Rules of Backgammon](https://www.fgbradleys.com/rules/Backgammon_Rules.pdf)

Notes on MCTS from Professor Gold (i can't link them)

[Covers the way of calculating nodes for MCTS in a game of chance](https://arxiv.org/pdf/0909.0801.pdf) (Chapter 4)

[Goes over different ways of tailoring data for an ANN](https://www.cs.cornell.edu/boom/2001sp/Tsinteris/gammon.htm) (This one recommends an input size of **197**)

[Brief summary of TD-Gammon](http://satirist.org/learn-game/systems/gammon/td-gammon.html)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# As you will see later in the code, we created a NN network
# In order to avoid retraining it everytime, we trained it once and saved it
# But this is completely optional so you don't have to run this cell

import tensorflow as tf

model = tf.keras.models.load_model('/content/drive/MyDrive/DS340/BackgammonNN') # this is the NN model that we trained and saved in a google drive file for ease of re-use


In [ ]:
import numpy as np
import random 
import math

# The following are the indices in our backgammon array that store the respective information

WHITE_BEARING = 25
BLACK_BEARING = 0
WHITE_OUT = 26
BLACK_OUT = 27
WHITE_TURN = 28
BOARD_SPLICE = slice(1, 25)
starting_board = np.zeros(29, dtype=int)

#Creates a starting board with the traditional piece positions, also randomly selects a starting player
def starting_board():
  #white_turn = random.random() > 0.5
  return np.array([0,2,0,0,0,0,-5,0,-3,0,0,0,5,-5,0,0,0,3,0,5,0,0,0,0,-2,0,0,0,True])

#Rolls die, handles the case of double rolls. Also allows you to reroll in case
#of doubles by setting first_turn=True (which is necessary on the first turn)
def roll(first_turn=False):
  roll_1 = random.randint(1,6)
  roll_2 = random.randint(1,6)
  if roll_1 == roll_2:
    if first_turn:
      return roll(first_turn=True)
    return np.full(4,roll_1)
  else:
    return np.array([roll_1,roll_2])

#Function that gives all possible single moves given a board state and set of rolls
def possible_moves(board, rolls):
    #options is an array of all possible moves given as:
    #(new board_state (array), string clarifying move taken (str), roll used to make that move (int))
    options = []
    #Creates the variable can_bear, which is true only if the conditions for bearing 
    #for the current player are met
    if board[WHITE_TURN]:
      can_bear = not np.any([x > 0 for x in board[1:19] + [board[WHITE_OUT]]])
    else:
      can_bear = not np.any([x < 0 for x in board[7:25] + [board[BLACK_OUT]]])
      
    #Branch of logic assuming white players turn
    if board[WHITE_TURN]:
        #Branch of logic assuming there is a piece that needs to be brought into the game
        if board[WHITE_OUT]:
            for roll in rolls:
                #only considers option if the space you rolled to is a valid landing spot
                if board[roll] >= -1:
                  possible_move = board.copy()
                  if possible_move[roll] == -1:
                    possible_move[roll] = 0
                    possible_move[BLACK_OUT] += 1
                  possible_move[roll] += 1
                  possible_move[WHITE_TURN] = len(rolls) != 1 
                  possible_move[WHITE_OUT] -= 1
                  options.append((possible_move, f'Bring in white piece to position {roll}',roll))

        #Assuming no pieces need to be brought back on board
        else:
          #Iterate over every space on the board
          for ind, space in enumerate(board[BOARD_SPLICE]):
            #position used instead of index since board is one_indexed 
            pos = ind + 1
            #checking only spaces where there are white pieces (value at space > 0)
            if space > 0:
              #checking possible move for piece given the rolls
              for roll in rolls:
                #creates option of moving piece to totally empty space
                if (pos + roll) < 25 and board[pos + roll] >= 0:
                  possible_move = board.copy()
                  possible_move[pos + roll] += 1
                  possible_move[WHITE_TURN] = len(rolls) != 1 
                  possible_move[pos] -= 1
                  options.append((possible_move, f'Move piece {roll} steps from position {pos} to position {pos + roll}.',roll))
                #create option if the space has a blot to be captured
                elif (pos + roll) < 25 and board[pos + roll] == -1:
                  possible_move = board.copy()
                  possible_move[pos + roll] = 1
                  possible_move[BLACK_OUT] += 1
                  possible_move[WHITE_TURN] = len(rolls) != 1 
                  possible_move[pos] -= 1
                  options.append((possible_move, f'Move piece {roll} steps from position {pos} to capture opponents piece at {pos + roll}.',roll))

                #create option if the player can legally bear and has rolled a high enough number to bear
                elif can_bear and (pos + roll) >= 25:
                  #small logic block due to specific rules of bearing
                  #i.e. you can only bear a piece where space != roll if no higher value pieces can be moved
                  if pos == 19:
                    no_alt = True
                  else:
                    no_alt = np.all([x < 1 for x in board[19:pos]])
                  #To bear off a piece, you must be exactly (roll) steps from goal
                  #or this is the 
                  if roll + pos == 25 or no_alt:
                    possible_move = board.copy()
                    possible_move[WHITE_BEARING] += 1
                    possible_move[WHITE_TURN] = len(rolls) != 1 
                    possible_move[pos] -= 1
                    options.append((possible_move, f'Bear off piece from position {pos} using {roll} steps.',roll))

    #The below code block is identical to the above, but is from the perspective
    #of the black piece player
    else:
        if board[BLACK_OUT]:
            for roll in rolls:
                if board[25-roll] <= 1:
                  possible_move = board.copy()
                  if possible_move[25-roll] == 1:
                    possible_move[25-roll] = 0
                    possible_move[WHITE_OUT] += 1
                  possible_move[25-roll] -= 1
                  possible_move[WHITE_TURN] = len(rolls) == 1 
                  possible_move[BLACK_OUT] -= 1
                  options.append((possible_move, f'Bring in black piece to position {roll}',roll))

        else:
          for ind, space in enumerate(board[BOARD_SPLICE]):
            pos = ind + 1
            if space < 0:
              for roll in rolls:
                #Check if the space is empty
                if (pos - roll) > 0 and board[pos - roll] <= 0:
                  possible_move = board.copy()
                  possible_move[pos - roll] -= 1
                  possible_move[WHITE_TURN] = len(rolls) == 1 
                  possible_move[pos] += 1
                  options.append((possible_move, f'Move piece {roll} steps from position {pos} to position {pos - roll}.',roll))
                #Check if the space has a blot to be captured
                elif (pos - roll) > 0 and board[pos - roll] == 1:
                  possible_move = board.copy()
                  possible_move[pos - roll] = -1
                  possible_move[WHITE_OUT] += 1
                  possible_move[WHITE_TURN] = len(rolls) == 1 
                  possible_move[pos] += 1
                  options.append((possible_move, f'Move piece {roll} steps from position {pos} to capture opponents piece at {pos - roll}.',roll))
                #check if the player can bear off their piece
                elif can_bear and (pos - roll) <= 0:
                  if pos == 6:
                    no_alt = True
                  else:
                    no_alt = np.all([x > -1 for x in board[pos+1:7]])
                  if pos - roll == 0 or no_alt:
                    possible_move = board.copy()
                    possible_move[BLACK_BEARING] += 1
                    possible_move[WHITE_TURN] = len(rolls) == 1 
                    possible_move[pos] += 1
                    options.append((possible_move, f'Bear off piece from position {pos} using {roll} steps.',roll))

    #Handles the case where no moves are available to the player   
    if not options:
      new_board = board
      new_board[WHITE_TURN] = not board[WHITE_TURN]
      return [(new_board, "No possible moves", None)]
    
    return options

#Function for giving the set of moves that use all given rolls for a user
def full_step(board, rolls):
  #branch handling a four roll turn
  if len(rolls) == 4:
    #creates branching tree of all possible 4 move combinations by calling
    #possible_moves recursively on its own children
    curr_turn = board[WHITE_TURN]
    rolls = np.asarray(rolls[0:2])
    options = possible_moves(board, rolls)
    for i in range(3):
      if options[0][2] == None:
        break
      new_options = []
      for option in options:
        next_moves = possible_moves(option[0], rolls)
        move_descriptions = [option[1] + "\nThen " + x[1].lower() for x in next_moves]
        new_options += [(next_moves[i][0], move_descriptions[i], next_moves[i][2]) for i in range(len(next_moves))]
      options = new_options

    #removes duplicate boards from the set before returning them as a list
    seen_boards = [[0]*29]
    filtered_options = []
    for ind, val in enumerate(options):
      if val[0].tolist() not in seen_boards:
        seen_boards.append(val[0].tolist())
        filtered_options.append(val)

    for option in filtered_options:
        option[0][WHITE_TURN] = not curr_turn


    return filtered_options

  else:
    #For cases of two dice rolls, calls possible_moves twice
    #first using both rolls as the input, then again using only the roll not
    #used by the program in each case as the input
    options = possible_moves(board, rolls)
    curr_turn = board[WHITE_TURN]

    full_options = []
    for option in options:
      if options[0][2] == None:
        full_options = options
        break
      rem_roll = [x for x in rolls if x != option[2]]
      second_moves = possible_moves(option[0], rem_roll)
      move_descriptions = [option[1] + " Then " + x[1] for x in second_moves]
      full_options += [(second_moves[i][0], move_descriptions[i], second_moves[i][2]) for i in range(len(second_moves))]

    #removes duplicate moves
    seen_boards = [[0]*29]
    filtered_options = []
    for ind, val in enumerate(full_options):
      if val[0].tolist() not in seen_boards:
        seen_boards.append(val[0].tolist())
        filtered_options.append(val)

    for i in filtered_options:
      i[0][WHITE_TURN] = not curr_turn

    return filtered_options

In [ ]:
# this block of code calculates the probability of getting each possible number from a dice roll 
# the result is stored in an array for later use

roll_dist = [0] * 12
for i in range(1,7):
  for j in range(1,7):
    if i == j:
      for k in range(1,5):
        if i*k-1 < 13:
          roll_dist[i*k-1] += 1
    else:
      roll_dist[i-1] += 1
      roll_dist[j-1] += 1
      roll_dist[i+j-1] += 1
roll_dist = [x/36 for x in roll_dist]

In [ ]:
# This cell contains the entirety of our MCTS

def winner_score(board):

    if board[WHITE_BEARING] == 15:
        return 2000

    if board[BLACK_BEARING] == 15:
        return -2000

class ChanceNode: # holds chance nodes

    def __init__(self, board): # takes in the board 

        self.board = board # assigns the board
        self.chance = True # declares that this is a chance node


    def __equal__(self, other): # this compares if two chance nodes are equal 

        return other.chance and other.board == self.board


class DecisionNode: # holds decision nodes

    def __init__(self, board, rolls): # similar to chance nodes but it also holds the roll that got it here

        self.board = board
        self.chance = False # declares that this is not a chance node
        self.rolls = rolls # the rolls that brought it here

    def __equal__(self, other): # compares two decision nodes and ensures the same roll brought them to the same board state

        return other.chance and other.board == self.board and self.rolls == other.rolls


def sample(tree, T_chance, T_decision, chance_mapper, V, m = 10, model = None, white_turn = True): # sampling method which does both exploration and expansion

    if not m: # if you don't have any more layers to discover then return

        return 0

    elif tree.chance: # if this is a chance node

        rolled = roll() # find a random roll
        r = 0 # for now, the reward for this roll is the sum of the dice normalized
        hor = DecisionNode(tree.board, rolled) # build the decision node for this roll
        if hor not in T_decision: # if it isn't already in T, then add it to it

            T_decision[hor] = 0 

        reward = r + sample(hor, T_chance, T_decision, chance_mapper, V, m-1, model, not white_turn) # find the net reward by going down a layer
    
    elif not T_decision.get(tree,0): # if it is an unexplored decision node then explore it 

        T_decision[tree] = 0
        reward = rollout(1, tree.board) # expects the reward from the rollout function

    else: # if it an explored decision node

        if game_over(tree.board):

            reward = winner_score(tree.board)

        else:
            a = select_action(tree, T_chance, T_decision, chance_mapper, V, m,white_turn = white_turn) # find the board for the corresponding best action
            if str(a) not in chance_mapper:

                ha = ChanceNode(a) # make a chance node out of this
                chance_mapper[str(a)] = ha

            else:

                ha = chance_mapper[str(a)]
            reward = sample(ha, T_chance, T_decision, chance_mapper, V, m) # carry out the rest from the chance node


    T = T_chance if tree.chance else T_decision # declare T based on tree's type to make next line simpler
    V[tree] = (reward + T.get(tree, 0) * V.get(tree, 0))/(T.get(tree, 0) + 1) # update the value of the utility of this node    
    T[tree] = T.get(tree, 0) + 1 # increment the number of times we have seen this node
    return reward # return the net reward


def select_action(tree, T_chance, T_decision, chance_mapper, V, m , C = math.sqrt(2), white_turn = True): # takes in a decision tree and returns the best action

    A = full_step(tree.board, tree.rolls) # returns the possibilities from this roll

    U = [] # stores the unexplored children
    for a in A: # for every turn for this roll check if it has been seen

        if str(a[0]) in chance_mapper:
          
          ha = chance_mapper[str(a[0])]
        
        else:

          ha = ChanceNode(a[0]) # construct the chance node
          chance_mapper[str(a[0])] = ha

        if not T_chance.get(ha, 0): # if it has not been explored, then just append it

            U.append(a[0])

    if U: # if there are unexplored nodes then return a random one

        a = random.choice(U)
        return a

    else: #otherwise find the one with the best UCB value
        
        max_node = A[0][0]
        max_val = float('-inf')

        for a in A: # go over all the explored nodes

            if str(a[0]) in chance_mapper:
          
                ha = chance_mapper[str(a[0])]
                
            else:

                ha = ChanceNode(a[0]) # construct the chance node
                chance_mapper[str(a[0])] = ha

            val = V[ha]/(2*m) + C * math.sqrt(math.log(T_decision[tree])/ T_chance[ha]) # using formula from the research paper
            if white_turn and val > max_val:

                max_val = val
                max_node = a[0]

            if not white_turn and val < max_val:

                max_val = val
                max_node = a[0]

        return max_node # return board with max value


TIME = 250 # this variable is the spread factor on our MCTS algorithm
def pUCT(board, roll, legal_moves, iterations):

  T_chance = {} # this is the T dict for chance nodes
  T_decision = {} #this is the T dict for decision nodes
  V = {} # this is the utility dict for all nodes
  chance_mapper = {}

  start_node = DecisionNode(board, roll)
  for i in range(TIME):
    sample(start_node, T_chance, T_decision, chance_mapper, V, iterations)
  # We look for the start node that has the most playouts -
  # not win % because this way favors nodes that have been tried quite a bit
  # (and are also good, or they wouldn't have been tried
  highest_utility = (0,float('-inf'))
  for i in range(len(legal_moves)):

    #x = ChanceNode(legal_moves[i][0])
    x = chance_mapper.get(str(legal_moves[i][0]), None)
    if V.get(x,float('-inf')) > highest_utility[1]:
      highest_utility = (i, V.get(x,float('-inf')))    

  return legal_moves[highest_utility[0]]

def rollout(choice, board):

    if choice == 0 :

        return NN_rollout(board)

    else:

        return heuristic(board)

def NN_rollout(board, model = model):
  board, state = transform_board(board)
  b = np.asarray([board])
  s = np.asarray([state])
  pred = model.predict([b,s], verbose=0)
  if node.board[WHITE_TURN] == 1:
    pred = 1-pred
  return pred


def heuristic(board):
  #check for boards in a won state
  if board[WHITE_BEARING] == 15:
    return 2000

  if board[BLACK_BEARING] == 15:
      return -2000

  #final total score, value for spaces occupied, spaces with two blocks, beared off pieces and pieces out
  score = 0
  OCC_VAL = 1
  PROT_VAL = 3
  BEAR_VAL = 35
  OUT_VAL = -10
  ATTACK_VAL = -6

  score += BEAR_VAL*board[WHITE_BEARING]
  score -= BEAR_VAL*board[BLACK_BEARING]
  score += OUT_VAL*board[WHITE_OUT]
  score -= OUT_VAL*board[BLACK_OUT]
  #counter for number of two stacks in a row
  counter = 0

  for ind, val in enumerate(board[BOARD_SPLICE]):
    pos = ind + 1
    attack_prob = 0
    if val == 1:
      for dist, prob in enumerate(roll_dist):
        if pos+dist+1 > 24:
          break
        if board[pos + dist+1] < 0:
          attack_prob += prob
      score += attack_prob * ATTACK_VAL

    if val == -1:
      for dist, prob in enumerate(roll_dist):
        if pos-dist - 1 < 1:
          break
        if board[pos - dist - 1] > 0:
          attack_prob += prob
      score -= attack_prob * ATTACK_VAL

    #if space has white piece
    if val > 0:
      #checking if there is a four prime or greater for black
      if counter < -3:
        score -= counter * counter

      #resets counter if prime is of insufficient length
      if counter < 0: counter = 0

      #if there is a stack of two or greater
      if val > 1:
        score += OCC_VAL * PROT_VAL
        counter += 1
      #for single pieces
      else:
        score += OCC_VAL
        if counter > 3:
          score += counter * counter
        counter = 0
      score += pos*val
    #black version of all above code
    elif val < 0:
      if counter > 3:
        score += counter * counter
      if counter > 0: counter = 0

      if val < -1:
        counter -= 1
        score -= OCC_VAL * PROT_VAL

      else:
        score -= OCC_VAL

        if counter < -3:
          score -= counter * counter

        counter = 0
    
    #if space is empty, reset prime counter
    else:
      counter = 0  

      score -= (24-pos) * val
  return score

        

In [ ]:
class BoardPrinter: # this class just has a string function that is used to print the board in a reasonable fashion
    def __init__(self, board):

        self.board = board #array representation of board state    

    def __str__(self):
      
        answer = " "
        for num in range(1,25):

            answer += str(num) + " "

        rows = max(max(self.board), abs(min(self.board)))
        temp = self.board.copy()
        for row in range(rows):
          
            answer += '\n'
            counter = 1
            for i in range(1, 25):
            
                answer += " " * len(str(counter)) 
                if temp[i] == 0:

                    answer += ' '
                
                elif temp[i] > 0:

                    answer += 'X'
                    temp[i] -= 1

                elif temp[i] < 0:

                    answer += 'O'
                    temp[i] += 1

                counter += 1
                

        answer += '\n'
        WHITE_BEARING = 25
        BLACK_BEARING = 0
        WHITE_OUT = 26
        BLACK_OUT = 27
        answer += f'White pieces beared off: {self.board[WHITE_BEARING]}'
        answer += '\n'
        answer += f'Black pieces beared off: {self.board[BLACK_BEARING]}'
        answer += '\n'
        answer += f'White pieces out: {self.board[WHITE_OUT]}'
        answer += '\n'
        answer += f'Black pieces out: {self.board[BLACK_OUT]}'
        answer += '\n'
        return answer

In [ ]:
# This cell sets up the code that allows a person to play a game with the AI

import sys

def game_over(board): # as soon one of the players has all of their pieces beared off then they win

  return board[BLACK_BEARING] == 15 or board[WHITE_BEARING] == 15


def find_winner(board): 

    ''' Assumes that the game is over
    Return which player won the game'''

    if board[BLACK_BEARING] == 15:

        return 'Black'

    else:

        return 'White'


def play_move(board, index, roll, pieces, mult): # modifies the board based on the move

  board[index - roll] -= mult*pieces
  board[index] += mult*pieces

def print_rolls(rolls): # prints the rolls 

  printout = 'Dice rolls are '
  for roll in rolls:

    printout += str(roll) + ' '

  return print(printout)


def get_player_move(board, rolls, moves): # this move gets the player move

    '''Provides a set of options of possible moves to the user and gets them to choose'''
    print('Here is a list of possible moves: ')    
    counter = 1 # checks how many options they have

    for move in moves: # go over the different options and print them for the user

        print(f'Option {counter} is :') 
        print(BoardPrinter(move[0]))
        print(move[1])
        print()
        counter += 1

    choice = 0 # stores the user choice
    while True: # this makes sure the user makes a legal move
        

        choice = input('Which move do you prefer? ')
        if choice == 'quit':

            sys.exit('Game has been ended')

        if not choice.isnumeric() or not 0 < int(choice) < counter:

            print('Please enter a valid option number')
            continue

        choice = int(choice)
        break

    return moves[choice-1] # send the chosen choice back


  

def play():
    
    board = starting_board() # find the starting board
    first_turn = True # it is the first_turn
    white_turn = True # we assume white starts but this can be changed later
    while not game_over(board): # while the game isn't over keep playing
        # White turn (AI)
        rolls = roll(first_turn) # find the rolls
        first_turn = False # it can't be the first roll after a roll
        print_rolls(rolls) # print the rolls
        print(BoardPrinter(board)) # print the board

        if white_turn: # computer_turn
          legal_moves = full_step(board, rolls)
          if len(legal_moves) > 1 or legal_moves[0][1] != "No possible moves":  # there are legal moves
              print("Thinking...")
              best_move = pUCT(board, rolls, legal_moves, 5) # finds the best move
              board = best_move[0] # the board is the best move according to the computer
              print(best_move[1])
              print()
          else: # if they're no moves, then it gets skipped
              print("White has no legal moves; skipping turn...")
              board[WHITE_TURN] = not board[WHITE_TURN]
        
        else: # human_turn

          legal_moves = full_step(board, rolls) # find all the moves they can do
          if len(legal_moves) > 1 or legal_moves[0][1] != "No possible moves":
              best_move = get_player_move(board, rolls, legal_moves) # get the player and modifies the board likewise
              board = best_move[0] 
          else:
              print("Black has no legal moves; skipping turn...")
              board[WHITE_TURN] = not board[WHITE_TURN]


        white_turn = not white_turn # change the turn

    winner = find_winner(board) # find the winner of the board since the game is over
    if winner == 'White': # print the respective winner
        print("White won!")
    elif winner == "Black":
        print("Black won!")


In [ ]:
# Run this to play a game against our AI!

play()

Dice rolls are 3 6 
 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
 X         O   O          X  O           X     X              O
 X         O   O          X  O           X     X              O
           O   O          X  O           X     X               
           O              X  O                 X               
           O              X  O                 X               
White pieces beared off: 0
Black pieces beared off: 0
White pieces out: 0
Black pieces out: 0

Thinking...
Move piece 3 steps from position 12 to position 15. Then Move piece 6 steps from position 17 to position 23.

Dice rolls are 6 3 
 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
 X         O   O          X  O     X     X     X           X  O
 X         O   O          X  O           X     X              O
           O   O          X  O                 X               
           O              X  O                 X               
           O                 O       

KeyboardInterrupt: ignored

#Data collection and training for NN

MCTS complex Heuristic:  
T=100,m=25: 94% on 50 games  
T =250, m=25: 96% on 100 games

Only complex heuristic:

N = 500: 95%

MCTS Vs Heuristic:

N= 100, 55% of the times MCTS won


In [ ]:
# this is a function that returns the best move according to the heuristic

def bestmove(legal_moves, white_turn = True):

    best = legal_moves[0] # the first one is the base one ot compare to
    best_rollout = heuristic(best[0]) # rollout value is its heuristic value

    for move in legal_moves: # go over all the moves and compare it

        temp = heuristic(move[0]) # get its heuristic value
        if white_turn and temp > best_rollout: # white wants to maximize the heuristic score

            best = move
            best_rollout = temp

        if not white_turn and temp< best_rollout: # black wants to minimize it

            best = move
            best_rollout = temp

    return best # return the best


In [ ]:
def player_chooser(player, board, rolls, legal_moves, depth, white_turn):

    if player == 'MCTS':

        return pUCT(board, rolls, legal_moves, depth) 

    if player == 'Heuristic':

        return bestmove(legal_moves, white_turn)

    if player == 'Random':

        return random.choice(legal_moves)

In [ ]:

def play_enhanced(player1, player2, depth): # plays a game between AIs of your choice
    
    board = starting_board() # find the starting board
    first_turn = True # it is the first_turn
    white_turn = True # we assume white starts but this can be changed later
    while not game_over(board): # while the game isn't over keep playing

        # White turn (AI one)
        rolls = roll(first_turn) # find the rolls
        first_turn = False # it can't be the first roll after a roll
        if white_turn: # computer_turn
          legal_moves = full_step(board, rolls)
          if len(legal_moves) > 1 or legal_moves[0][1] != "No possible moves":   # (list is non-empty)

              best_move = player_chooser(player1, board, rolls, legal_moves, depth, True)
              board = best_move[0] # the board is the best move according to the computer
          else: # if they're no moves, then it gets skipped

              board[WHITE_TURN] = not board[WHITE_TURN]

        #Black turn(AI two) --- similar code as the white turn
        else: 

          legal_moves = full_step(board, rolls) # find all the moves they can do
          if len(legal_moves) > 1 or legal_moves[0][1] != "No possible moves":  

              best_move = player_chooser(player2, board, rolls, legal_moves, depth, False) 
              board = best_move[0] 

          else: 

              board[WHITE_TURN] = not board[WHITE_TURN]


        white_turn = not white_turn # change the turn

    winner = find_winner(board) # find the winner of the board since the game is over
    if winner == 'White': # print the respective winner
        return 1
    elif winner == "Black":
        return 0

# The following piece of code is only here for one to play the game many times to see which AI wins more 

wins = 0 
GAMES = 100
for i in range(GAMES):

    wins += play_enhanced('Heuristic', 'Heuristic', 5)
    print(i+1)
    print(wins/(i+1))

print(wins/GAMES)

In [ ]:
# The following play function is different from the above enhanced_play in that it also returns all the boards visited during the game for training the NN

def data_play(player1, player2, depth): # plays a game between AIs of your choice and returns all the boards of your choice
    
    boards = []
    board = starting_board() # find the starting board
    first_turn = True # it is the first_turn
    white_turn = True # we assume white starts but this can be changed later
    while not game_over(board): # while the game isn't over keep playing

        # White turn (AI one)
        rolls = roll(first_turn) # find the rolls
        first_turn = False # it can't be the first roll after a roll
        if white_turn: # computer_turn
          legal_moves = full_step(board, rolls)
          if len(legal_moves) > 1 or legal_moves[0][1] != "No possible moves":   # (list is non-empty)

              best_move = player_chooser(player1, board, rolls, legal_moves, depth, True)
              board = best_move[0] # the board is the best move according to the computer
          else: # if they're no moves, then it gets skipped

              board[WHITE_TURN] = not board[WHITE_TURN]

        #Black turn(AI two) --- similar code as the white turn
        else: 

          legal_moves = full_step(board, rolls) # find all the moves they can do
          if len(legal_moves) > 1 or legal_moves[0][1] != "No possible moves":  

              best_move = player_chooser(player2, board, rolls, legal_moves, depth, False) 
              board = best_move[0] 

          else: 

              board[WHITE_TURN] = not board[WHITE_TURN]

        boards.append(board)
        white_turn = not white_turn # change the turn

    winner = find_winner(board) # find the winner of the board since the game is over
    if winner == 'White': # print the respective winner
        return boards, 1
    elif winner == "Black":
        return boards, 0

#Data storage/preprocessing

In [ ]:
"""function that turns standard board states into boards with feature-extraction 
to emphasize the significance of stacking pieces, and clarify the distinction between
positive and negative numbers as black/white pieces to the AI 
"""
def transform_board(board):
  board = list(board)
  game_state = [board[0]] + board[25:]

  board_states = np.zeros((24, 4, 1))
  for ind, val in enumerate(board[BOARD_SPLICE]): # segregates each item on the board based on the number of pieces on that space
    if val > 1:
      board_states[ind][0][0] = val
    elif val == 1:
      board_states[ind][1][0] = val
    elif val == -1:
      board_states[ind][2][0] = val
    else:
      board_states[ind][3][0] = val
    
  return board_states, game_state

In [ ]:
NUM = 2 # number of iterations of random games for NN data
import pandas as pd

import numpy as np

def my_read_csv(): # this reads from the csv

    df = pd.read_csv('/content/drive/MyDrive/DS340/data.csv') # convert the csv into a dataframe
    boards = [] # create 3 arrays for each of the different columns
    wins = []
    total = []

    for i in range(len(df)): # go over the dataframe

        temp = df.iloc[i,0].split('.') # split the string into numbers
        temp = [int(i) for i in temp] # put those numbers into an array
        boards.append(temp) # add the array to array of boards
        wins.append(df.iloc[i,1]) 
        total.append(df.iloc[i,2])

    return boards, wins, total # return all three arrays


def data_creator(num = NUM): # this function augments existing data in the csv with another num simulations of the game
    
    boards, wins, total = my_read_csv() # the csv stores the board state, the number of times white won from that state, the total games that this board state came up in 
    trainer = dict() # this is the dictionary that will store the existing results

    for i in range(len(boards)): # go over the csv data and throw it in the dictionary
        
        board = boards[i]
        str_rep = '' # this will hold the string representation of the board state
        for item in board:

            str_rep += '.' + str(item)
        trainer[str_rep[1:]] = [wins[i], total[i]] 

    for i in range(num): # do the simulations
        boards, winner = data_play('MCTS', 'Random', 5) # choose which types of AI you want to see play

        for board in boards: # go over the boards created in the process

            str_rep = '' # this will hold the string representation
            for item in board:

                str_rep += '.' + str(item)

            existing = trainer.get(str_rep[1:], [0,0]) # see if this board already exists
            existing[0] += winner # add the information of wins
            existing[1] += 1
            trainer[str_rep[1:]] = existing # put the information back

    df = pd.DataFrame.from_dict(data = trainer, orient = 'index') # turn this into a dataframe

    df.to_csv('data.csv') # converty the dataframe into a csv
    !cp data.csv "drive/My Drive/" # upload the csv


def convert(): # this converts the csv into two arrays of board states and wins 

    boards, wins, total = my_read_csv()
    wins = np.asarray(wins)
    total = np.asarray(total)
    wins = wins/total
  
    return boards, wins

boards, wins = convert()

In [ ]:
data_creator(10000)

#Tensorflow

In [ ]:
#modifies default board states into states useable by CNN
mod_boards, states, y = [], [], []
for ind, board in enumerate(boards):
  x1, x2 = transform_board(board)
  mod_boards.append(x1)
  states.append(x2)
  y.append(wins[ind])

mod_boards = np.asarray(mod_boards)
states = np.asarray(states)
y = np.asarray(y)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, ReLU, Dropout, Flatten, Dense, concatenate

#Framework for CNN model
board = Input((24, 4, 1))
play_state = Input((5,))

conv_1 = Conv2D(32, (12,2),strides=(1,2), activation="relu")(board)
drop_1 = Dropout(0.2)(conv_1)
conv_2 = Conv2D(64, kernel_size=(2,2), activation="relu")(drop_1)
drop_2 = Dropout(0.2)(conv_2)
flattened = Flatten()(drop_2)

dense_1 = Dense(128, activation="relu")(flattened)
dropout_1 = Dropout(0.2)(dense_1)

dense_2 = Dense(64, activation="relu")(dropout_1)
dropout_2 = Dropout(0.2)(dense_2)

dense_3 = Dense(8, activation="relu")(dropout_2)
dropout_3 = Dropout(0.2)(dense_3)

merged = concatenate([dropout_3, play_state])

output = Dense(1, activation="sigmoid")(merged)
model = keras.models.Model(inputs=(board,play_state), outputs=output)

model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, ReLU, Dropout, Flatten, Dense, concatenate

#Framework for fully connected network
input = Input((29,))

dense_1 = Dense(128, activation="relu")(input)
dropout_1 = Dropout(0.2)(dense_1)

dense_2 = Dense(64, activation="relu")(dropout_1)
dropout_2 = Dropout(0.2)(dense_2)

dense_3 = Dense(8, activation="relu")(dropout_2)
dropout_3 = Dropout(0.2)(dense_3)

output = Dense(1, activation="sigmoid")(dropout_3)
model = keras.models.Model(inputs=(input), outputs=output)

model.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping
earlyStop = EarlyStopping("val_loss",patience=4)
model.fit(x=[mod_boards,states],y=y,epochs=30, validation_split=0.1,callbacks=[earlyStop])

Epoch 1/30
27285/27285 [==============================] - 248s 9ms/step - loss: 0.1434 - val_loss: 0.1407
Epoch 2/30
27285/27285 [==============================] - 242s 9ms/step - loss: 0.1388 - val_loss: 0.1433
Epoch 3/30
27285/27285 [==============================] - 241s 9ms/step - loss: 0.1374 - val_loss: 0.1429
Epoch 4/30
27285/27285 [==============================] - 259s 9ms/step - loss: 0.1362 - val_loss: 0.1419
Epoch 5/30
27285/27285 [==============================] - 307s 11ms/step - loss: 0.1351 - val_loss: 0.1448


In [ ]:
#saving best version of the models for future use
model.save("/content/drive/MyDrive/DS340/BackgammonNN")

#Testing

The following three code cells are us just testing our code 

In [ ]:
#just testing the above functions dont mind me:
#roll()
for i in range(3):
  print(roll())
print("*" * 15)

#starting_board
board = starting_board()
boardNode = BoardPrinter(board)
print(boardNode)
print(f"Whites turn is {board[WHITE_TURN]}")
rolls = roll()
print(f"Rolls are {rolls}")
print(board[BOARD_SPLICE])
print("*" * 15)

moves = possible_moves(board,rolls)

hit_board = [0,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
test_kill = possible_moves(hit_board,[1])

bear_board = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,1]
test_bear = possible_moves(bear_board,[3,1])

respawn_board = [0,-2,-1,-2,0,-2,-2,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1]
test_respawn = possible_moves(respawn_board, [4,2])

#test on simple starting board
print("Testing simple board:")
for move in moves[:3]:
  print(move[1])
  print(move[0][BOARD_SPLICE])
  print()
print("*" * 15)

#test hitting a piece and ending turn
print("Testing hitting piece and turn change:")
for move in test_kill:
  print(move[1])
  print(move[0])
  print()
print("*" * 15)

print("Testing bearing off pieces:")
for move in test_bear:
  print(move[1])
  print(move[0][BOARD_SPLICE])
  print()
print("*" * 15)

print("Testing respawning pieces:")
for move in test_respawn:
  print(move[1])
  print(move[0][BOARD_SPLICE])
  print()

[1 5]
[6 2]
[2 5]
***************
 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 
 X         O   O          X  O           X     X              O
 X         O   O          X  O           X     X              O
           O   O          X  O           X     X               
           O              X  O                 X               
           O              X  O                 X               
White pieces beared off: 0
Black pieces beared off: 0
White pieces out: 0
Black pieces out: 0

Whites turn is 1
Rolls are [2 3]
[ 2  0  0  0  0 -5  0 -3  0  0  0  5 -5  0  0  0  3  0  5  0  0  0  0 -2]
***************
Testing simple board:
Move piece 2 steps from position 1 to position 3.
[ 1  0  1  0  0 -5  0 -3  0  0  0  5 -5  0  0  0  3  0  5  0  0  0  0 -2]

Move piece 3 steps from position 1 to position 4.
[ 1  0  0  1  0 -5  0 -3  0  0  0  5 -5  0  0  0  3  0  5  0  0  0  0 -2]

Move piece 2 steps from position 12 to position 14.
[ 2  0  0  0  0 -5  0 -3  0  0  0  4 -5

In [ ]:
#testing my call to full_step to see if it provides all possible combinations and accurate text descriptions
mult_options = [0] * 29
mult_options[3] = 1
mult_options[5] = 1
mult_options[WHITE_TURN] = 1
mult_options = np.array(mult_options)

x = full_step(mult_options, [3,3,3,3])

print(mult_options[BOARD_SPLICE])
for val in x:
  print(val[1])
  print(val[0][BOARD_SPLICE])
  print("*" * 15)

[0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Move piece 3 steps from position 3 to position 6.
Then move piece 3 steps from position 5 to position 8.
Then move piece 3 steps from position 6 to position 9.
Then move piece 3 steps from position 8 to position 11.
[0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
***************
Move piece 3 steps from position 3 to position 6.
Then move piece 3 steps from position 5 to position 8.
Then move piece 3 steps from position 6 to position 9.
Then move piece 3 steps from position 9 to position 12.
[0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
***************
Move piece 3 steps from position 3 to position 6.
Then move piece 3 steps from position 5 to position 8.
Then move piece 3 steps from position 8 to position 11.
Then move piece 3 steps from position 11 to position 14.
[0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
***************
Move piece 3 steps from position 3 to position 6.
Then move piece 3 steps from position 6 to position 

In [ ]:
start = [0,-2,2,0,0,0,4,0,2,0,0,0,-5,5,0,0,0,-2,0,-4,0,0,-2, 0,2,0,0,0,0]
start[WHITE_TURN] = 1
hey =   [0,-2,2,0,0,0,4,0,2,0,0,0,-5,5,0,0,0,-2,0,-3,0,0,-1,-2,2,0,0,0,0]
hi =    [0,-2,2,0,0,0,4,0,2,0,0,0,-4,5,0,0,0,-3,0,-4,0,0,-2, 0,2,0,0,0,0]
heuristic(hey)
heuristic(hi)

157